# **INTRODUCTION**
> Welcome to the April edition of the 2022 Tabular Playground Series! This month's challenge is a time series classification problem.
    You've been provided with thousands of sixty-second sequences of biological sensor data recorded from several hundred participants who could have been in either of two possible activity states.

**DATA DESCRIPTION**

In this competition, you'll classify 60-second sequences of sensor data, indicating whether a subject was in either of two activity states for the duration of the sequence.

*Files and Field Descriptions*
1.  train.csv - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
    * sequence - a unique id for each sequence
    * subject - a unique id for the subject in the experiment
    * step - time step of the recording, in one second intervals
    * sensor_00 - sensor_12 - the value for each of the thirteen sensors at that time step
2. train_labels.csv - the class label for each sequence.
    * sequence - the unique id for each sequence.
    * state - the state associated to each sequence. This is the target which you are trying to predict.
3. test.csv - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.
4. sample_submission.csv - a sample submission file in the correct format.

# **DATA EXPLORATION**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train= pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train.csv")
train.head()


In [ ]:
print("dataset shape:\n",train.shape)
print("*********************************************************************************************************************************")
print("dataset stats:\n",train.describe())
print("*********************************************************************************************************************************")
print("dataset info:\n",train.info())
print("*********************************************************************************************************************************")
print("null values:\n",train.isnull().sum())
print("*********************************************************************************************************************************")
print("drop duplicates:\n",train.drop_duplicates())
print("*********************************************************************************************************************************")
print("After dropping duplicates shape of dataset:\n",train.shape)

In [ ]:
train_labels=pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv")
train_labels.head()

In [ ]:
print("dataset shape:\n",train_labels.shape)
print("*********************************************************************************************************************************")
print("dataset stats:\n",train_labels.describe())
print("*********************************************************************************************************************************")
print("dataset info:\n",train_labels.info())
print("*********************************************************************************************************************************")
print("null values:\n",train_labels.isnull().sum())
print("*********************************************************************************************************************************")
print("drop duplicates:\n",train_labels.drop_duplicates())
print("*********************************************************************************************************************************")
print("After dropping duplicates shape of dataset:\n",train_labels.shape)

In [ ]:
test=pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/test.csv")
test.head()

In [ ]:
print("dataset shape:\n",test.shape)
print("*********************************************************************************************************************************")
print("dataset stats:\n",test.describe())
print("*********************************************************************************************************************************")
print("dataset info:\n",test.info())
print("*********************************************************************************************************************************")
print("null values:\n",test.isnull().sum())
print("*********************************************************************************************************************************")
print("drop duplicates:\n",test.drop_duplicates())
print("*********************************************************************************************************************************")
print("After dropping duplicates shape of dataset:\n",test.shape)

In [ ]:
sample=pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv")
print("dataset shape:\n",sample.shape)
print("*********************************************************************************************************************************")
sample.head()

**As you can see that sample submission shape: (12218,2) and test shape: (733080, 16), we need to remove the duplicated sequence. We are keeping the last value of the sequence to remove duplicates.** 

In [ ]:
print("unique values:\n", test.sequence.nunique()/test.shape[0])
test.sort_values("sequence",inplace=True)
test.drop_duplicates("sequence",keep="last",inplace=True)
print("shape:\n",test.shape)
print("After removing duplicates unique values:\n",test.sequence.nunique()/test.shape[0])

**The test dataset has same size  as sample submission.**

# **DATA PRE-PROCESSING**

In [ ]:
train_labels.state.value_counts()

In [ ]:
train.subject.value_counts()

In [ ]:
# Identifying outliers
sns.boxplot(train["subject"])

In [ ]:
sensor=train[['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12']]
sensor.plot(subplots=True, figsize=(8, 8));

In [ ]:
plt.figure(figsize=(16, 16))
sensor.plot.box(vert=False)

In [ ]:
lower=np.where(sensor<=-600) 
upper=np.where(sensor>600)
print("old shape:",train.shape)
print("****************************************************************************************************")
train.drop(upper[0],inplace=True)
train.drop(lower[0],inplace=True)
print("new shape:",train.shape)

In [ ]:
plt.figure(figsize=(10, 10))
matrix = sensor.corr()
sns.heatmap(matrix,cmap="BuPu")
plt.show()

# **BIDIRECTIONAL LSTM**
 A Bidirectional LSTM, or biLSTM, is a sequence processing model that consists of two LSTMs: one taking the input in a forward direction, and the other in a backwards direction. BiLSTMs effectively increase the amount of information available to the network, improving the context available to the algorithm.

![Bidirectional LSTM](https://miro.medium.com/max/1400/0*ZsTT3zzTNGF-6OsR.jpg)

Generally in normal LSTM network we take output directly as shown in first figure but in bidirectional LSTM network output of forward and backward layer at each stage is given to activation layer which is a neural network and output of this activation layer is considered. This output contains the information or relation of past and future word also.

In [ ]:
data=pd.merge(right=train,left=train_labels,how="left",on="sequence")
data.head()

In [ ]:
print("shape:",data.shape)
data.isnull().sum()

In [ ]:
sequence_length = 10
def generate_data(X,y, sequence_length = 100, step = 200):
    X_local = []
    y_local = []
    for start in range(0, len(data) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(data.loc[:, "sensor_00":"sensor_12"].values, data.state)


In [ ]:
X_sequence.shape, y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.metrics import binary_accuracy

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(100,return_sequences=True,input_shape=[X_sequence.shape[1], X_sequence.shape[2]])))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_sequence, y, batch_size=64, epochs=5)
print("********************************************************************************************************************")
print(model.summary())

In [ ]:
training_size = int(len(X_sequence) * 0.7)
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [ ]:
history=model.fit(X_train, y_train, batch_size=64, epochs=500)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('model loss vs accuracy')
plt.xlabel('epoch')
plt.legend(['loss', 'accuracy'], loc='upper right')
plt.show() 

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_test_pred = model.predict(X_test)

# **PREDICTION**

In [ ]:
test.shape

In [ ]:
sequence_length = len(test)
x_test = test.loc[:, ['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12']]
x_test = x_test.squeeze()
x_test = x_test.to_numpy()
X_final= np.reshape(x_test, (-1, sequence_length, x_test.shape[1]))

In [ ]:
X_final.shape

In [ ]:
y_final=model.predict(X_final)

In [ ]:
y_final = (y_final[:] > 0.5).astype(int)
y_final = np.reshape(y_final, (-1,))
y_final

# **SUBMISSON**

In [ ]:
sub=pd.DataFrame({'sequence':sample["sequence"],"state":y_final})
sub.to_csv('submission.csv', index=False)
sub.head(3)